## Amazon AgentCore Bedrock Code Interpreter を使った高度なデータ分析 - チュートリアル (Strands)

このチュートリアルでは、Python を使ってコードを実行することで高度なデータ分析を行う AI エージェントを作成する方法を示します。LLM によって生成されたコードを実行するために、Amazon Bedrock AgentCore Code Interpreter を使用します。

このチュートリアルでは、AgentCore Bedrock Code Interpreter を使って以下のことを実演します。
1. サンドボックス環境のセットアップ
2. ユーザーのクエリに基づいてコードを生成し、高度なデータ分析を行うストランドベースのエージェントの構成
3. Code Interpreter を使ってサンドボックス環境でコードを実行
4. 結果をユーザーに表示

## 前提条件
- Bedrock AgentCore Code Interpreter にアクセスできる AWS アカウント
- コードインタープリターリソースを作成および管理するための必要な IAM 許可
- 必要な Python パッケージがインストールされている (boto3、bedrock-agentcore、strands を含む)
- Amazon Bedrock でモデルを呼び出す許可を持つ IAM ロール
 - US オレゴン (us-west-2) リージョンの Claude 3.7 Sonnet モデルにアクセスできる (Strands SDK のデフォルトモデル)

## IAM 実行ロールには、以下の IAM ポリシーが付与されている必要があります

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

日本語訳:
"バージョン": "2012-10-17",
"ステートメント": [
    {
        "効果": "許可",
        "アクション": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "リソース": "*"
    },
    {
        "効果": "許可", 
        "アクション": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "リソース": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
~~~

## 動作の仕組み

コード実行サンドボックスは、コード インタプリター、シェル、ファイル システムを備えた分離された環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。大規模言語モデルがツールの選択を支援した後、コードがこのセッション内で実行され、その結果がユーザーまたはエージェントに合成のために返されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、Code Interpreter クライアントを初期化しましょう。

デフォルトのセッションタイムアウトは 900 秒(15 分)です。しかし、データの詳細な分析を行うため、今回はセッションタイムアウト時間を 1200 秒(20 分)に長めに設定してセッションを開始します。

In [ ]:
!pip install --upgrade -r requirements.txt

In [110]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
from strands import Agent, tool
import json
import pandas as pd
from typing import Dict, Any, List

# AWS の対応リージョン内で Code Interpreter を初期化します。

日本語訳:
code_client = CodeInterpreter('us-west-2')
code_client.start(session_timeout_seconds=1200)

'01K01J7Z3DH445CJ0YW14BK2GY'

## 2. ローカルデータファイルの読み込み

ここでは、サンプルデータファイルの内容を読み込みます。このファイルには、 Name 、 Preferred_City 、 Preferred_Animal 、 Preferred_Thing の 4 列があり、ランダムなデータが約 30 万件記録されています。

後ほど、このファイルを agent を使って分析し、分布とアウトライアを把握します。

In [111]:
df_data = pd.read_csv("samples/data.csv")
df_data.head()

,Name,Preferred_City,Preferred_Animal,Preferred_Thing
0,Betty Ramirez,Dallas,Elephant,Sofa
1,Jennifer Green,Naples,Bee,Shirt
2,John Lopez,Helsinki,Zebra,Wallet
3,Susan Gonzalez,Beijing,Chicken,Phone
4,Jennifer Wright,Buenos Aires,Goat,Wallet


In [ ]:
def read_file(file_path: str) -> str:
    """ファイルの内容を読み込むためのヘルパー関数 (エラー処理付き)"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

data_file_content = read_file("samples/data.csv")

## 3. Sandbox 環境のためのファイルの準備

Sandbox 環境で作成したいファイルを定義する構造を作成します。

In [113]:
files_to_create = [
                {
                    "path": "data.csv",
                    "text": data_file_content
                }]

## 4. Creating Helper Function for Tool Invocation

This helper function will make it easier to call sandbox tools and handle their responses. Within an active session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [114]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    以下が日本語訳になります。

"""サンドボックスツールを呼び出すためのヘルパー関数

    引数:
        tool_name (str): 呼び出すツールの名前
        arguments (Dict[str, Any]): ツールに渡す引数

    戻り値:
        Dict[str, Any]: JSON 形式の結果
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Code Sandbox にデータファイルを書き込む

次に、データファイルをサンドボックス環境に書き込み、正常に作成されたことを確認します。

In [115]:
# Write files to sandbox

日本語訳:
# サンドボックスにファイルを書き込む

The `write_file` function allows you to write data to a file in the sandbox. 指定した `path` にファイルが存在しない場合は新規作成され、存在する場合は上書きされます。

```python
import sandbox

data = "Hello World!"
sandbox.write_file("/path/to/hello.txt", data)
```

`binary` フラグを `True` に設定すると、バイナリモードでファイルが開かれます。これは、バイナリデータ (画像など) を扱う場合に便利です。

```python
import sandbox

binary_data = b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a"  # PNG ヘッダー
sandbox.write_file("/path/to/image.png", binary_data, binary=True)
```

ファイルの内容は、`read_file` 関数を使って読み取ることができます。
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created

ファイルが作成されたことを確認する
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 1 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}], "isError": false}


## 6. Strands ベースのエージェントを使用した高度な分析の実行

ここでは、サンドボックス (上記) にアップロードしたデータファイルに対してデータ分析を実行するエージェントを構成します。

### 6.1 システムプロンプトの定義
AI アシスタントの振る舞いと機能を定義します。私たちは、アシスタントに常にコードの実行とデータに基づく推論によって回答を検証するよう指示しています。

In [116]:
SYSTEM_PROMPT = はい、以下のように翻訳しました。

"""あなたは、提供されたツールを使ってコードの実行によりすべての回答を検証する、役立つAIアシスタントです。ツールを使わずに質問に答えないでください。

検証の原則:
1. コード、アルゴリズム、または計算について主張する場合は、それらを検証するコードを書きます
2. execute_python を使って数学的な計算、アルゴリズム、論理を テストします
3. 回答を出す前に、テストスクリプトを作成して理解度を検証します  
4. 実際のコード実行で作業過程を必ず示します
5. 不確かな場合は、制限を明示的に述べ、検証できる範囲を検証します

アプローチ:
- プログラミングの概念について尋ねられた場合は、それをコードで実装して示します
- 計算を求められた場合は、プログラムで計算し、コードも示します
- アルゴリズムを実装する場合は、正しさを証明するためのテストケースを含めます
- 透明性を保つため、検証プロセスを文書化します  
- サンドボックスでは実行間で状態が維持されるので、前の結果を参照できます

使用可能なツール:
- execute_python: Python コードを実行して出力を確認

応答フォーマット: execute_pythonツールは以下のフィールドを含むJSONレスポンスを返します:
- sessionId: サンドボックスセッションID
- id: リクエストID 
- isError: エラーがあったかどうかを示すブール値
- content: type と text/data を持つコンテンツオブジェクトの配列
- structuredContent: コード実行の場合、stdout、stderr、exitCode、executionTimeが含まれます

コード実行が成功した場合、出力は content[0].text と structuredContent.stdout の両方に表示されます。
isError フィールドを確認して、エラーがあったかどうかを確認してください。

できる限り徹底的で正確に、検証可能な場合は必ず回答を検証してください。"""

技術的な用語はそのまま残し、半角英数字の前後に半角スペースを挿入しました。

### 6.2 コード実行ツールの定義
次に、エージェントがコードサンドボックスでコードを実行するためのツールとして使用する関数を定義します。 @tool デコレータを使って、関数をエージェントのカスタムツールとしてアノテートします。

アクティブなコード インタープリター セッション内では、サポートされている言語 (Python、JavaScript) でコードを実行し、依存関係の構成に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [120]:
#Define and configure the code interpreter tool

#コード解釈ツールを定義および構成する

日本語訳:

コード 解釈ツールを 定義 および 構成する ことを 説明します 。 半角英数字 の 前後 には 半角スペース を 挿入し 、 コード 、 コマンド 、 変数名 、 関数名 など の 技術的 な 用語 は そのまま 残します 。
@tool
def execute_python(code: str, description: str = "") -> str:
    翻訳するテキスト:
"""Execute Python code in the sandbox."""

日本語訳:
"""サンドボックス内で Python コードを実行します。"""

    if description:
        code = f"# {description}の説明\n{code}のコード

    #Print generated Code to be executed

生成されたコードを出力して実行します。
    print(f"\n Generated Code: {code}")


    # Call the Invoke メソッドを呼び出し、初期化されたコード インタープリター セッション内で生成されたコードを実行する

日本語訳:
# Invoke メソッドを呼び出し、初期化されたコードインタプリターセッション内で生成されたコードを実行します
    response = code_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
    for event in response["stream"]:
        return json.dumps(event["result"])

### 6.3 エージェントの設定
Strands SDK を使用してエージェントを作成し、設定します。上で定義したシステムプロンプトとツールを提供し、コード生成を実行します。

日本語訳:
### 6.3 エージェントの設定
Strands SDK を使用して エージェント を作成し、設定します。システムプロンプト と、上で定義した ツール を提供し、コード生成 を実行します。

In [121]:
#configure the strands agent including the tool(s)

日本語訳:

# strands エージェントとツール (s) を構成します。
agent=Agent(
        tools=[execute_python],
        system_prompt=SYSTEM_PROMPT,
        callback_handler=None)

## 7. エージェントの呼び出しと応答処理
私たちはクエリでエージェントを呼び出し、エージェントの応答を処理します

注意: 非同期実行には非同期環境での実行が必要です

## 7.1 探索的データ分析 (EDA) を実行するためのクエリ

日本語訳:

探索的データ分析 (EDA) は、データセットの特性を理解するための重要なステップです。 SQL を使用して EDA を実行することができます。以下は、 EDA を実行するための一般的な SQL クエリの例です。

データの概要を取得するには、 `SELECT COUNT(*) FROM table_name;` を使用して行数を確認します。

列の値の分布を確認するには、 `SELECT column_name, COUNT(*) FROM table_name GROUP BY column_name;` を使用します。

欠損値の有無を確認するには、 `SELECT SUM(CASE WHEN column_name IS NULL THEN 1 ELSE 0 END) AS null_count FROM table_name;` を使用します。

数値データの概要統計量 (最小値、最大値、平均値、中央値など) を取得するには、 `SELECT MIN(column_name), MAX(column_name), AVG(column_name), MEDIAN(column_name) FROM table_name;` を使用します。

カテゴリデータの一意の値とその出現回数を確認するには、 `SELECT column_name, COUNT(*) FROM table_name GROUP BY column_name;` を使用します。

これらのクエリは、データセットの構造と内容を理解するのに役立ちます。探索的データ分析の結果に基づいて、データの前処理や特徴量エンジニアリングの必要性を判断できます。

以下が日本語訳になります。

コード サンドボックス 環境のデータ ファイルに対して探索的データ分析を実行するよう エージェントに指示するクエリから始めましょう。

In [123]:
query = "Perform exploratory data analysis(EDA) on the file 'data.csv'. Tell me about distributions and outlier values."

# Invoke the agent asynchcronously and stream the response

エージェントを非同期で呼び出し、レスポンスをストリーミングする
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response

以下のように翻訳します。

# テキストレスポンスをストリーミング

半角英数字の前後に半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

I'll perform an exploratory data analysis (EDA) on the data.csv file to examine distributions and identify any outliers. Let me work through this step by step.I see we're missing some libraries. Let me try without seaborn and use the standard libraries:Now let's analyze each categorical column in more detail:Let's check for patterns in preferences and analyze distributions visually:Let's check for outliers and get more detailed statistics:Let's further explore interesting patterns and relationships in the data:Based on the comprehensive exploratory data analysis (EDA) of the data.csv file, here's a summary of the findings:

## 1. Dataset Overview

- **Size**: The dataset contains 299,130 records with 4 columns: Name, Preferred_City, Preferred_Animal, and Preferred_Thing.
- **Data types**: All columns contain categorical (object) data.
- **Unique values**:
  - 1,722 unique names (combinations of first and last names)
  - 55 unique cities
  - 50 unique animals
  - 51 unique things

## 2.

## 7.2 情報を抽出するクエリ

さて、コード サンドボックス環境のデータファイルから特定の情報を抽出するよう、エージェントに指示しましょう。

In [122]:
query = "Within the file 'data.csv', how many individuals with the first name 'Kimberly' have 'Crocodile' as their favourite animal?"

# Invoke the agent asynchcronously and stream the response

エージェントを非同期で呼び出し、レスポンスをストリーミングする
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response

テキストの応答をストリーミングします。

import asyncio
import websockets

async def hello(websocket, path):
    name = await websocket.recv()
    print(f"< {name}")

    greeting = f"Hello {name}!"

    await websocket.send(greeting)
    print(f"> {greeting}")

start_server = websockets.serve(hello, "localhost", 8765)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

I'll help you analyze the data.csv file to find individuals named 'Kimberly' who have 'Crocodile' as their favorite animal. Let me use Python to check this data file.
 Generated Code: # Checking if data.csv exists and examining its structure
# First, let's check if the file exists
import os
import pandas as pd

# Check if the file exists
if os.path.exists('data.csv'):
    print(f"File 'data.csv' exists. Reading the file...")
    # Try to read the CSV file
    try:
        df = pd.read_csv('data.csv')
        print("File successfully loaded. Here's a preview:")
        print(df.head())
        print(f"\nTotal rows in the file: {len(df)}")
        print(f"Columns in the file: {df.columns.tolist()}")
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print(f"File 'data.csv' does not exist in the current directory.")
    print(f"Current directory contents: {os.listdir()}")
Great! The data.csv file exists and has been successfully loaded. Now I can see it has

## 8. クリーンアップ

最後に、Code Interpreter セッションを停止してクリーンアップを行います。セッションの使用が終わったら、リソースを解放し、不要な課金を避けるためにセッションを停止する必要があります。

In [109]:
# コードインタープリターセッションを停止する

 session = PromptSession(cli=cli)
 try:
     session.app()
 except KeyboardInterrupt:
     print('無事終了しました')
 finally:
     session.exit()
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!
